# Generating demographic summary statistics

In [23]:
from video_face_detection import *

In [24]:
# face_detection
face_analyzer = VideoDetect()
face_analysis = face_analyzer.main(task='face_detection')

Start Job Id: 76485d0de4ac14e5ad54b0b403474c8d482a6c4c464a3f4ed8a0cc87b1c9dd86
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
....................
.....76485d0de4ac14e5ad54b0b403474c8d482a6c4c464a3f4ed8a0cc87b1c9dd86
SUCCEEDED
Matching Job Found:76485d0de4ac14e5ad54b0b403474c8d482a6c4c464a3f4ed8a0cc87b1c9dd86
h264
19044
QuickTime / MOV
29.9700298309326

h264
19044
QuickTime / MOV
29.970029830932617
Face: {'BoundingBox': {'Width': 0.15861114859580994, 'Height': 0.41504377126693726, 'Left': 0.407012403011322, 'Top': 0.18365830183029175}, 'AgeRange': {'Low': 35, 'High': 53}, 'Smile': {'Value': False, 'Confidence': 91.69222259521484}, 'Eyeglasses': {'Value': False, 'Confidence': 97.87004089355469}, 'Sunglasses': {'Value': False, 'Confidence': 99.8916015625}, 'Gender': {'Value': 'Male', 'Confidence': 98.81217956542969}, 'Beard': {'Value': False, 'Confidence': 99.62757873535156}, 'Mustache': {'Value': False, 'Confidence': 99.97607421875}, 'EyesOpen': {'Value': True, 'Confidence': 99.8568344116211}, 'MouthOpen': {'Value': False, 'Confidence': 93.48116302490234}, 'Emotions': [{'Type': 'CALM', 'Confidence': 70.41114044189453}, {'Type': 'CONFUSED', 'Confidence': 0.0}, {'Type': 'HAPPY', 'Confidence': 1.9374687671661377}, {'Type': 'SAD', 'Confidence': 1.4188629388809204}, {'Type': 'DISGUSTED', 'Confidence': 8.45333194732666}, {'Type': 'SURPRISED'

h264
19044
QuickTime / MOV
29.970029830932617
Face: {'BoundingBox': {'Width': 0.049604035913944244, 'Height': 0.13053052127361298, 'Left': 0.7299239039421082, 'Top': 0.15783391892910004}, 'AgeRange': {'Low': 38, 'High': 59}, 'Smile': {'Value': False, 'Confidence': 94.6181411743164}, 'Eyeglasses': {'Value': False, 'Confidence': 99.9278564453125}, 'Sunglasses': {'Value': False, 'Confidence': 99.99527740478516}, 'Gender': {'Value': 'Male', 'Confidence': 98.54544067382812}, 'Beard': {'Value': False, 'Confidence': 99.24911499023438}, 'Mustache': {'Value': False, 'Confidence': 99.97188568115234}, 'EyesOpen': {'Value': True, 'Confidence': 97.70502471923828}, 'MouthOpen': {'Value': False, 'Confidence': 92.026611328125}, 'Emotions': [{'Type': 'CALM', 'Confidence': 78.51760864257812}, {'Type': 'HAPPY', 'Confidence': 1.7943007946014404}, {'Type': 'CONFUSED', 'Confidence': 0.0}, {'Type': 'SAD', 'Confidence': 0.8546140789985657}, {'Type': 'SURPRISED', 'Confidence': 3.3626575469970703}, {'Type': 'DI

h264
19044
QuickTime / MOV
29.970029830932617
Face: {'BoundingBox': {'Width': 0.0453248992562294, 'Height': 0.12732742726802826, 'Left': 0.7277345657348633, 'Top': 0.1598108559846878}, 'AgeRange': {'Low': 35, 'High': 52}, 'Smile': {'Value': False, 'Confidence': 97.56319427490234}, 'Eyeglasses': {'Value': False, 'Confidence': 99.99993896484375}, 'Sunglasses': {'Value': False, 'Confidence': 100.0}, 'Gender': {'Value': 'Male', 'Confidence': 85.56421661376953}, 'Beard': {'Value': False, 'Confidence': 99.13148498535156}, 'Mustache': {'Value': False, 'Confidence': 99.9857177734375}, 'EyesOpen': {'Value': False, 'Confidence': 98.81114959716797}, 'MouthOpen': {'Value': False, 'Confidence': 90.80247497558594}, 'Emotions': [{'Type': 'DISGUSTED', 'Confidence': 9.777656555175781}, {'Type': 'SAD', 'Confidence': 7.383230209350586}, {'Type': 'CALM', 'Confidence': 65.60327911376953}, {'Type': 'HAPPY', 'Confidence': 3.568906307220459}, {'Type': 'ANGRY', 'Confidence': 7.569860458374023}, {'Type': 'SURPR

h264
19044
QuickTime / MOV
29.970029830932617
Face: {'BoundingBox': {'Width': 0.05408036708831787, 'Height': 0.13425956666469574, 'Left': 0.15501415729522705, 'Top': 0.1644764095544815}, 'AgeRange': {'Low': 48, 'High': 68}, 'Smile': {'Value': False, 'Confidence': 89.24131774902344}, 'Eyeglasses': {'Value': False, 'Confidence': 99.99988555908203}, 'Sunglasses': {'Value': False, 'Confidence': 99.99999237060547}, 'Gender': {'Value': 'Male', 'Confidence': 99.68303680419922}, 'Beard': {'Value': True, 'Confidence': 65.9251480102539}, 'Mustache': {'Value': False, 'Confidence': 97.49230194091797}, 'EyesOpen': {'Value': False, 'Confidence': 81.90928649902344}, 'MouthOpen': {'Value': False, 'Confidence': 57.62575912475586}, 'Emotions': [{'Type': 'HAPPY', 'Confidence': 5.502258777618408}, {'Type': 'CALM', 'Confidence': 4.3932881355285645}, {'Type': 'CONFUSED', 'Confidence': 0.0}, {'Type': 'DISGUSTED', 'Confidence': 14.219503402709961}, {'Type': 'SAD', 'Confidence': 41.615352630615234}, {'Type': '

h264
19044
QuickTime / MOV
29.970029830932617
Face: {'BoundingBox': {'Width': 0.14538517594337463, 'Height': 0.3828623592853546, 'Left': 0.38284987211227417, 'Top': 0.16079233586788177}, 'AgeRange': {'Low': 48, 'High': 68}, 'Smile': {'Value': True, 'Confidence': 53.33660888671875}, 'Eyeglasses': {'Value': False, 'Confidence': 97.55479431152344}, 'Sunglasses': {'Value': False, 'Confidence': 99.71410369873047}, 'Gender': {'Value': 'Male', 'Confidence': 99.92623901367188}, 'Beard': {'Value': True, 'Confidence': 97.20555114746094}, 'Mustache': {'Value': False, 'Confidence': 82.23309326171875}, 'EyesOpen': {'Value': True, 'Confidence': 99.9935302734375}, 'MouthOpen': {'Value': False, 'Confidence': 80.04821014404297}, 'Emotions': [{'Type': 'SURPRISED', 'Confidence': 0.5469613671302795}, {'Type': 'DISGUSTED', 'Confidence': 11.469666481018066}, {'Type': 'SAD', 'Confidence': 9.914360046386719}, {'Type': 'CALM', 'Confidence': 3.7987008094787598}, {'Type': 'ANGRY', 'Confidence': 16.67729568481445

done


In [ ]:
face_analysis

In [ ]:
# person_tracking
person_analyzer = VideoDetect()
person_analyzer.main(task='person_tracking')